# Instructions
This notebook allows you to load and evaluate a huggingface model on a subset of BLiMP (a linguistic acceptability judgment dataset) and GLUE (a natural language understanding benchmark collection). It is HIGHLY recommended to clone the GitHub repository and evaluate your model in the command-line; this will give you more freedom in the kinds of models you can evaluate. However, Colab provides a GPU that will allow you to load and evaluate smaller models.

To use this notebook:

1. Start by making a copy of this notebook so that you can make edits and run the code: File > Save a copy in Drive.

2. Set Runtime > Change runtime type > Hardware accelerator to GPU if it isn't already.

3. Run the setup script to install the required packages for evaluating.

4. Upload your model to the colab in the `/content/model_folder/` directory. This folder should include the following files, and probably a couple more depending on the type of model and tokenizer you use:
* `config.json`
* `pytorch_model.bin`
* `tokenizer_config.json`
* `vocab.json`

  a. To obtain these files given your pre-trained model and your tokenizer, load them using huggingface `transformers` and save them using these commands:
```
tokenizer.save_pretrained("./model_dir")
model.save_pretrained("./model_dir")
```
  b. Then, upload all the contents of `model_dir` (including any other files not mentioned above) to the `model_folder` folder in this Colab.

5. Choose the proper model type in the dropdown in the "load model and evaluate" cell. Use "decoder" for autoregressive (sometimes called "causal") language models, like GPT/OPT; "encoder" for masked language models, like BERT/RoBERTa; or "encoder-decoder" for text-to-text models, like T5/BART.

6. Run the cells below to load and evaluate your model.

In [ ]:
#@title Setup script { display-mode: "form" }
#@markdown Run this cell to install the necessary packages (may take a few minutes).
%%shell
# Remove previous installation if it exists
cd /content
mkdir model_folder
pip uninstall -y lm-eval
rm -rf evaluation-pipeline/

# Install evaluation-pipeline
git clone -b colab https://github.com/babylm/evaluation-pipeline &> /dev/null
cd evaluation-pipeline/
pip install -e ".[colab]"
# Install other necessary packages
pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0 --extra-index-url https://download.pytorch.org/whl/cu113

# Unpack dataset
unzip filter_data.zip

Obtaining file:///content/evaluation-pipeline
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/huggingface/accelerate (to revision main) to /tmp/pip-install-j3v7u_pr/accelerate_f24beb81975a44ed9d6cf379ee598a09
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-install-j3v7u_pr/accelerate_f24beb81975a44ed9d6cf379ee598a09
  Resolved https://github.com/huggingface/accelerate to commit 85a75d4c3d0deffde2fc8b917d9b1ae1cb580eb2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 20.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 42.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [12]:
# GPT-2
model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left'
model.config.pad_token_id = tokenizer.pad_token_id

tokenizer.save_pretrained("./model_folder")
model.save_pretrained("./model_folder")

# Evaluation

In [13]:
#@title Load model and evaluate (BLiMP) { display-mode: "form" }
model = "/content/model_folder/" #@param {"type": "string"}
model_type = "decoder" #@param ["decoder", "encoder", "encoder-decoder"]
# file_name = "examples3.csv" #@param {"type": "string"}
# model_names = ["opt-125m", "opt-350m", "opt-1.3b", "opt-2.7b"] #@param {"type": "raw"}

%cd /content/evaluation-pipeline
%run /content/evaluation-pipeline/babylm_eval.py \
  "$model" \
  "$model_type" \
  -t "blimp"

/content/evaluation-pipeline


Generating train split: 0 examples [00:00, ? examples/s]


» Assigning unique IDs to 'blimp_from_file+null' docs


INFO:lm_eval.evaluator:
» Assigning unique IDs to 'blimp_from_file+null' docs


Map:   0%|          | 0/1956 [00:00<?, ? examples/s]

...


» Constructing 'blimp_from_file+null' contexts and requests


INFO:lm_eval.evaluator:
» Constructing 'blimp_from_file+null' contexts and requests
100%|██████████| 5535/5535 [00:00<00:00, 7058.70it/s]


» Running all `loglikelihood` requests



INFO:lm_eval.evaluator:
» Running all `loglikelihood` requests
100%|██████████| 11063/11063 [02:38<00:00, 69.71it/s]


subject_verb_agreement:	88.06%

Scores:
anaphor_agreement:	99.69%
argument_structure:	84.00%
binding:	78.94%
control_raising:	80.00%
determiner_noun_agreement:	95.92%
ellipsis:	85.10%
filler_gap:	80.97%
irregular_forms:	95.78%
island_effects:	78.33%
npi_licensing:	76.56%
quantifiers:	71.92%
subject_verb_agreement:	88.06%


In [8]:
#@title Load model and evaluate ((Super)GLUE) { display-mode: "form" }
#@markdown Run this cell to fine-tune your model on (Super)GLUE tasks.
#@markdown We provide some default hyperparameters that you may adjust.
model = "/content/model_folder/" #@param {"type": "string"}
learning_rate = 5e-5 #@param {"type": "number"}
batch_size = 64 #@param {"type": "integer"}
eval_every = 200 #@param {"type": "integer"}
patience = 10 #@param {"type": "integer"}
max_epochs = 10 #@param {"type": "integer"}
seed = 12 #@param {"type": "integer"}
# file_name = "examples3.csv" #@param {"type": "string"}
# model_names = ["opt-125m", "opt-350m", "opt-1.3b", "opt-2.7b"] #@param {"type": "raw"}

%cd /content/evaluation-pipeline
!./finetune_all_tasks.sh \
    "$model" \
    "$learning_rate" \
    "$patience" \
    "$batch_size" \
    "$eval_every" \
    "$max_epochs" \
    "$seed"

Streaming output truncated to the last 5000 lines.
[WARNING|tokenization_utils_base.py:3276] 2024-03-25 22:24:16,446 >> Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
[WARNING|tokenization_utils_base.py:3276] 2024-03-25 22:24:16,446 >> Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
[WARNING|tokenization_utils_base.py:3276] 2024-03-25 22:24:16,446 >> Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
[WARNING|tokenization_utils_base.py:3276] 2024-03-25 22:24:16,44